In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import time
import pickle
from tqdm import tqdm
os.chdir('/home/gebhart/projects/sheaf_kg')

import sheaf_kg.harmonic_extension as harmonic_extension

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pykeen
import torch
from pykeen.pipeline import pipeline

In [2]:
dataset = 'FB15k-237'
num_test = 100
model_name = 'StructuredEmbedding_1000epochs_64dim_SoftplusLossloss_1234seed_20210127-1707'
save_loc = '/home/gebhart/projects/sheaf_kg/data/{}/{}/trained_model.pkl'.format(dataset,model_name)
q2b_path = '/home/gebhart/projects/sheaf_kg/data/{}-q2b'.format(dataset)
# q2b_path = '/home/gebhart/projects/sheaf_kg/data/{}-betae'.format(dataset)

In [3]:
training_path = '/home/gebhart/projects/sheaf_kg/data/{}-q2b/train.txt'.format(dataset)
testing_path = '/home/gebhart/projects/sheaf_kg/data/{}-q2b/test.txt'.format(dataset)
validation_path = '/home/gebhart/projects/sheaf_kg/data/{}-q2b/valid.txt'.format(dataset)

In [4]:
result = pipeline(
        model='StructuredEmbedding',
        training=training_path,
        testing=testing_path,
        validation=validation_path,
        random_seed=1235,
        device='gpu',
        training_kwargs=dict(num_epochs=10),
        evaluation_kwargs=dict(),
        model_kwargs=dict(embedding_dim=64),
        training_loop='slcwa',
        loss='NSSA',
    )

model = result.model.to('cpu')
result.metric_results.to_df()

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...


INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=16.


INFO:pykeen.evaluation.evaluator:Evaluation took 277.02s seconds


,Side,Type,Metric,Value
0,head,avg,adjusted_mean_rank,0.631659
1,tail,avg,adjusted_mean_rank,0.629128
2,both,avg,adjusted_mean_rank,0.630394
3,head,avg,mean_rank,4509.776446
4,head,avg,mean_reciprocal_rank,0.014699
5,head,avg,hits_at_1,0.004232
6,head,avg,hits_at_3,0.011547
7,head,avg,hits_at_5,0.018568
8,head,avg,hits_at_10,0.033369
9,head,worst,mean_rank,4509.777816


In [5]:
query_structures = [('e', ('r', 'r')), ('e', ('r', 'r', 'r')), (('e', ('r',)), ('e', ('r',))), (('e', ('r',)), ('e', ('r',)), ('e', ('r',))), (('e', ('r', 'r')), ('e', ('r',))), ((('e', ('r',)), ('e', ('r',))), ('r',))]

query_name_dict = {('e',('r',)): '1p', 
                    ('e', ('r', 'r')): '2p',
                    ('e', ('r', 'r', 'r')): '3p',
                    (('e', ('r',)), ('e', ('r',))): '2i',
                    (('e', ('r',)), ('e', ('r',)), ('e', ('r',))): '3i',
                    ((('e', ('r',)), ('e', ('r',))), ('r',)): 'ip',
                    (('e', ('r', 'r')), ('e', ('r',))): 'pi',
                    (('e', ('r',)), ('e', ('r', 'n'))): '2in',
                    (('e', ('r',)), ('e', ('r',)), ('e', ('r', 'n'))): '3in',
                    ((('e', ('r',)), ('e', ('r', 'n'))), ('r',)): 'inp',
                    (('e', ('r', 'r')), ('e', ('r', 'n'))): 'pin',
                    (('e', ('r', 'r', 'n')), ('e', ('r',))): 'pni',
                    (('e', ('r',)), ('e', ('r',)), ('u',)): '2u-DNF',
                    ((('e', ('r',)), ('e', ('r',)), ('u',)), ('r',)): 'up-DNF',
                    ((('e', ('r', 'n')), ('e', ('r', 'n'))), ('n',)): '2u-DM',
                    ((('e', ('r', 'n')), ('e', ('r', 'n'))), ('n', 'r')): 'up-DM'
                }

In [6]:
ds = pykeen.datasets.get_dataset(dataset=dataset)
training = ds.training.mapped_triples
relid2label = ds.training.relation_id_to_label 
label2relid = {v:k for k,v in relid2label.items()}

entid2label = ds.training.entity_id_to_label 
label2entid = {v:k for k,v in entid2label.items()}

In [7]:
with open(os.path.join(q2b_path,'test-queries.pkl'), 'rb') as f:
    test_queries = pickle.load(f)

with open(os.path.join(q2b_path,'test-easy-answers.pkl'), 'rb') as f:
    test_answers = pickle.load(f)
    
with open(os.path.join(q2b_path,'id2rel.pkl'), 'rb') as f:
    id2rel = pickle.load(f)
    
with open(os.path.join(q2b_path,'id2ent.pkl'), 'rb') as f:
    id2ent = pickle.load(f)

In [8]:
def map_ent(e):
    return e

def map_rel(r):
    orientation = 1
    relname = id2rel[r]
    if relname[0] == '-':
        orientation = -1
    return r, orientation

In [9]:
def L_p(query, model):
    '''query of form ('e', ('r', 'r', ... , 'r')).
    here we assume 2 or more relations are present so 2p or greater
    '''
    ent = map_ent(query[0])
    invs = []
    rels = []
    for r in query[1]:
        mapped_id, orientation = map_rel(r) 
        rels.append(mapped_id)
        invs.append(orientation)
    n_path_ents = len(rels)
    
    edge_indices = np.concatenate([np.arange(0,n_path_ents)[:,np.newaxis].T, np.arange(1,n_path_ents+1)[:,np.newaxis].T], axis=0)
    
    rel_idx_tensor = torch.LongTensor(rels)
    left_restrictions = model.left_relation_embeddings(indices=rel_idx_tensor).view(-1, model.embedding_dim, model.embedding_dim).detach().numpy()
    right_restrictions = model.right_relation_embeddings(indices=rel_idx_tensor).view(-1, model.embedding_dim, model.embedding_dim).detach().numpy()
    
    restrictions = np.empty((len(rels), 2, left_restrictions.shape[2], left_restrictions.shape[1]))
    restrictions[:,0,:,:] = left_restrictions
    restrictions[:,1,:,:] = right_restrictions
#     for invix in range(len(invs)):
#         if invix == -1:
#             restrictions[:,0,:,:] = right_restrictions
#             restrictions[:,1,:,:] = left_restrictions
#         else:
#             restrictions[:,0,:,:] = left_restrictions
#             restrictions[:,1,:,:] = right_restrictions
    
    ent_idx_tensor = torch.LongTensor([ent])
    source_embeddings = model.entity_embeddings(indices=ent_idx_tensor).view(-1, model.embedding_dim).detach().numpy()
    
    B = np.array([0,n_path_ents],np.int)
    U = np.array(range(1,n_path_ents),np.int)
    source_vertices = [0]
    target_vertices = [1]
    LSchur = harmonic_extension.Kron_reduction(edge_indices, restrictions, B, U)
    return LSchur, source_vertices, target_vertices, source_embeddings

def L_i(query, model):
    '''query of form (('e', ('r',)), ('e', ('r',)), ... , ('e', ('r',)))'''
    num_intersects = len(query)
    ents = []
    rels = []
    invs = []
    for pair in query:
        ents.append(map_ent(pair[0]))
        rel, inv = map_rel(pair[1][0])
        rels.append(rel)
        invs.append(inv)
    n_ents = num_intersects
    edge_indices = np.concatenate([np.arange(0,n_ents)[:,np.newaxis].T, np.full(n_ents,n_ents)[:,np.newaxis].T], axis=0)
    
    rel_idx_tensor = torch.LongTensor(rels)
    left_restrictions = model.left_relation_embeddings(indices=rel_idx_tensor).view(-1, model.embedding_dim, model.embedding_dim).detach().numpy()
    right_restrictions = model.right_relation_embeddings(indices=rel_idx_tensor).view(-1, model.embedding_dim, model.embedding_dim).detach().numpy()
    
    restrictions = np.empty((len(rels), 2, left_restrictions.shape[2], left_restrictions.shape[1]))
    restrictions[:,0,:,:] = left_restrictions
    restrictions[:,1,:,:] = right_restrictions
#     for invix in range(len(invs)):
#         if invix == -1:
#             restrictions[:,0,:,:] = right_restrictions
#             restrictions[:,1,:,:] = left_restrictions
#         else:
#             restrictions[:,0,:,:] = left_restrictions
#             restrictions[:,1,:,:] = right_restrictions
    
    ent_idx_tensor = torch.LongTensor(ents)
    source_embeddings = model.entity_embeddings(indices=ent_idx_tensor).view(-1, model.embedding_dim).detach().numpy()
    
    L = harmonic_extension.Laplacian(edge_indices, restrictions)
    source_vertices = np.arange(n_ents)
    target_vertices = [n_ents]
    return L, source_vertices, target_vertices, source_embeddings

def L_ip(query, model):
    '''query of form ((('e', ('r',)), ('e', ('r',))), ('r',))'''
    ents = [map_ent(t[0]) for t in query[0]]
    rel0, inv0 = map_rel(query[0][0][1][0])
    rel1, inv1 = map_rel(query[0][1][1][0])
    rel2, inv2 = map_rel(query[1][0])
    rels = [rel0, rel1, rel2]
    invs = [inv0, inv1, inv2]
    n_ents = len(ents)
    edge_indices = np.array([[0,2],[1,2],[2,3]],np.int).T
    
    rel_idx_tensor = torch.LongTensor(rels)
    left_restrictions = model.left_relation_embeddings(indices=rel_idx_tensor).view(-1, model.embedding_dim, model.embedding_dim).detach().numpy()
    right_restrictions = model.right_relation_embeddings(indices=rel_idx_tensor).view(-1, model.embedding_dim, model.embedding_dim).detach().numpy()
    
    restrictions = np.empty((len(rels), 2, left_restrictions.shape[2], left_restrictions.shape[1]))
    restrictions[:,0,:,:] = left_restrictions
    restrictions[:,1,:,:] = right_restrictions
#     for invix in range(len(invs)):
#         if invix == -1:
#             restrictions[:,0,:,:] = right_restrictions
#             restrictions[:,1,:,:] = left_restrictions
#         else:
#             restrictions[:,0,:,:] = left_restrictions
#             restrictions[:,1,:,:] = right_restrictions
    
    ent_idx_tensor = torch.LongTensor(ents)
    source_embeddings = model.entity_embeddings(indices=ent_idx_tensor).view(-1, model.embedding_dim).detach().numpy()
    
    B = np.array([0,2,3],np.int)
    U = np.array([1],np.int)
    source_vertices = [0,1]
    target_vertices = [2]
    LSchur = harmonic_extension.Kron_reduction(edge_indices, restrictions, B, U)
    return LSchur, source_vertices, target_vertices, source_embeddings
    
def L_pi(query, model):
    '''query of form (('e', ('r', 'r')), ('e', ('r',)))'''
    ents = [map_ent(t[0]) for t in query]
    rel0, inv0 = map_rel(query[0][1][0])
    rel1, inv1 = map_rel(query[0][1][1])
    rel2, inv2 = map_rel(query[1][1][0])
    rels = [rel0, rel1, rel2]
    invs = [inv0, inv1, inv2]
    n_ents = len(ents)
    edge_indices = np.array([[0,2],[2,3],[1,3]],np.int).T
    
    rel_idx_tensor = torch.LongTensor(rels)
    left_restrictions = model.left_relation_embeddings(indices=rel_idx_tensor).view(-1, model.embedding_dim, model.embedding_dim).detach().numpy()
    right_restrictions = model.right_relation_embeddings(indices=rel_idx_tensor).view(-1, model.embedding_dim, model.embedding_dim).detach().numpy()
    
    restrictions = np.empty((len(rels), 2, left_restrictions.shape[2], left_restrictions.shape[1]))
    restrictions[:,0,:,:] = left_restrictions
    restrictions[:,1,:,:] = right_restrictions
#     for invix in range(len(invs)):
#         if invix == -1:
#             restrictions[:,0,:,:] = right_restrictions
#             restrictions[:,1,:,:] = left_restrictions
#         else:
#             restrictions[:,0,:,:] = left_restrictions
#             restrictions[:,1,:,:] = right_restrictions
    
    ent_idx_tensor = torch.LongTensor(ents)
    source_embeddings = model.entity_embeddings(indices=ent_idx_tensor).view(-1, model.embedding_dim).detach().numpy()
    
    B = np.array([0,1,3],np.int)
    U = np.array([2],np.int)
    source_vertices = [0,1]
    target_vertices = [2]
    LSchur = harmonic_extension.Kron_reduction(edge_indices, restrictions, B, U)
    return LSchur, source_vertices, target_vertices, source_embeddings

query_name_fn_dict = {'2p': L_p, '3p':L_p, '2i': L_i, '3i':L_i, 'ip':L_ip, 'pi': L_pi}

In [10]:
target_embeddings = model.entity_embeddings(indices=None).view(-1, model.embedding_dim).detach().numpy().T
allhits1 = []
allhits3 = []
allhits5 = []
allhits10 = []
allmrr = []
query_names = []
for query_structure in query_structures:
    print('Running query : {}'.format(query_structure))
    query_name = query_name_dict[query_structure]
    query_names.append(query_name)
    fn = query_name_fn_dict[query_name]
    hits1 = 0.
    hits3 = 0.
    hits5 = 0.
    hits10 = 0.
    mrr = 0.
    cnt = 0
    queries = list(test_queries[query_structure])
    for query in tqdm(queries[:num_test]):
        if len(test_answers[query]) > 0:
            # we have a non-trivial "easy" query
            answers = [map_ent(a) for a in test_answers[query]]
            L, source_vertices, target_vertices, source_embeddings = fn(query, model)
            Q = harmonic_extension.compute_costs(L,source_vertices,target_vertices,source_embeddings.flatten(),target_embeddings,source_embeddings.shape[1])
            
            sortd = np.sort(Q)
            idxleft = np.searchsorted(sortd, Q[answers], side='left') + 1 # optimistic ranking
            idxright = np.searchsorted(sortd, Q[answers], side='right') + 1 # pessimistic ranking
            
            hits1 += ((np.mean(idxleft <= 1) + np.mean(idxright <= 1)) / 2.)
            hits3 += ((np.mean(idxleft <= 3) + np.mean(idxright <= 3)) / 2.)
            hits5 += ((np.mean(idxleft <= 5) + np.mean(idxright <= 5)) / 2.)
            hits10 += ((np.mean(idxleft <= 10) + np.mean(idxright <= 10)) / 2.)
            mrr += ((np.mean(1./idxleft) + np.mean(1./idxright)) / 2.)
            cnt += 1
    if cnt > 0:
        allhits1.append(hits1/cnt)
        allhits3.append(hits3/cnt)
        allhits5.append(hits5/cnt)
        allhits10.append(hits10/cnt)
        allmrr.append(mrr/cnt)
    else:
        default = 0.
        allhits1.append(default)
        allhits3.append(default)
        allhits5.append(default)
        allhits10.append(default)
        allmrr.append(default)

  5%|▌         | 5/100 [00:00<00:02, 41.73it/s]

Running query : ('e', ('r', 'r'))


  4%|▍         | 4/100 [00:00<00:02, 36.47it/s]

Running query : ('e', ('r', 'r', 'r'))


  7%|▋         | 7/100 [00:00<00:01, 63.19it/s]

...

  3%|▎         | 3/100 [00:00<00:03, 27.37it/s]

Running query : (('e', ('r', 'r')), ('e', ('r',)))


  3%|▎         | 3/100 [00:00<00:04, 23.02it/s]

Running query : ((('e', ('r',)), ('e', ('r',))), ('r',))


100%|██████████| 100/100 [00:02<00:00, 36.46it/s]


In [11]:
cols = ['hits@1', 'hits@3', 'hits@5', 'hits@10', 'mrr']
df = pd.DataFrame(np.array([allhits1, allhits3, allhits5, allhits10, allmrr]).T, columns=cols, index=query_names) 

In [12]:
df * 100

,hits@1,hits@3,hits@5,hits@10,mrr
2p,0.000000,0.000000,0.004325,0.009734,0.041876
3p,0.001757,0.006013,0.012735,0.024563,0.047122
2i,0.007491,0.041111,0.046619,0.066130,0.075929
3i,0.000000,0.000000,0.000000,0.002332,0.044480
pi,0.000133,0.009645,0.020529,0.048685,0.070683
ip,0.000884,0.005138,0.022180,0.088220,0.060759
